In [138]:
import numpy as np
from sys import exit
from functools import reduce
import sympy
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt

In [123]:
def legendre_symbol(a, p):
    """ Compute the Legendre symbol a|p using
    Euler's criterion. p is a prime, a is
    relatively prime to p (if p divides
    a, then a|p = 0)
    Returns 1 if a has a square root modulo
    p, -1 otherwise.
    """
    ls = pow(a, (p - 1) // 2, p)
    return -1 if ls == p - 1 else ls

def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod
 
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

def stringToAscii(s):
    return [ord(c) for c in s]

def stringToBinary(s):
    return [bin(ord(c)) for c in s]

In [ ]:
msg = "Máté"
key = 580
print(msg)
asciiList = stringToAscii(msg)
binaryList = stringToBinary(msg)
print(binaryList)
k = ""
for j in binaryList:
    k+=j[2:]
print(list(k))
squareFreeCores = []
for j in range(key,key+len(k)):
    factors = sympy.primefactors(j)
    squareFreeFactorization = []
    for l in factors: 
        helper = j
        count = 0
        while helper%l==0:
            helper/=l
            count+=1
        if count%2==1:
            squareFreeFactorization.append(l)
    squareFreeCores+=squareFreeFactorization
primes = list(set(squareFreeCores))
primes.sort()
print(primes)
a=len(primes)
b=len(k)
print(a,b)
A = zeros(b,a)
for x in range(key,key+b):
    for y in primes:
        z = x
        cnt = 0
        while z % y == 0:
            z/=y
            cnt+=1
        if cnt%2 == 1:  
            A[x-key,primes.index(y)]=1
print("Solution exists",A.det()) ## We need a non-zero determinant!
msgBin = []
for x in k:
    msgBin.append(int(x))
print(msgBin)
m = Matrix(a,1,msgBin)
print(m)
A_inv = A.inv_mod(2) ## for some reason, this seems to be super slow
solution = A_inv*m
print(solution)
## Once the solution vector for the residues of the primes is established, then we just need to apply
## the Chinese Remainder Theorem for getting the final prime modulus (see paper)


Máté
['0b1001101', '0b11100001', '0b1110100', '0b11101001']
['1', '0', '0', '1', '1', '0', '1', '1', '1', '1', '0', '0', '0', '0', '1', '1', '1', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '0', '0', '1']
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 43, 53, 59, 67, 73, 83, 97, 101, 149, 151, 197, 199, 293, 587, 593, 599, 601, 607]
30 30
Solution exists -1
[1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1]
Matrix([[1], [0], [0], [1], [1], [0], [1], [1], [1], [1], [0], [0], [0], [0], [1], [1], [1], [1], [0], [1], [0], [0], [1], [1], [1], [0], [1], [0], [0], [1]])
